In [ ]:
%pip install ollama
%pip install langchain chromadb gradio 
%pip install -U langchain-community
%pip install pymupdf

  Using cached ollama-0.4.7-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
Using cached ollama-0.4.7-py3-none-any.whl (13 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
  Using cached langchain-0.3.18-py3-none-any.whl.metadata (7.8 kB)
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
  Using cached langchain_text_splitters-0.3.6-py3-none-any.whl.metadata (1.9 kB)
  Using cached langsmith-0.3.8-py3-none-any.whl.metadata (14 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl.metadata (4.2 kB)
  Using cached build-1.2.2.post1-py3-none-any.whl.

In [3]:
# Import necessary packages
import ollama  
import gradio as gr  # frontend

# Document processing and retrieval  
from langchain_community.document_loaders import PyMuPDFLoader  
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.vectorstores import Chroma  

# Embedding generation  
from langchain_community.embeddings import OllamaEmbeddings  

import re 

/Users/nekonyo/.pyenv/versions/3.10.12/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
response = ollama.chat(
    model="deepseek-r1:1.5b",  
    messages=[
        {"role": "user", "content": "who is naruto"}, 
    ],
)

In [10]:
print(response["message"]["content"])

<think>

</think>

Naruto is a popular Japanese manga and anime series created by Masashi Kishimoto. It follows the story of Naruto Uzumaki, a young ninja with a unique background and interests in various fields such as fantasy, science, and even philosophy. The series has been adapted into many films and TV shows, becoming one of the most widely recognized and beloved franchises in Japan and around the world.

Key elements of the series include:

1. **Naruto's Early Life**: Naruto is introduced at a young age in a small kingdom called Obanakashi, where he is taught various ninja skills by a wise mentor, Sasuke Uchiha.

2. **The Great战 ("Gigaseki no Na ga")**: Naruto faces challenges that test his abilities and knowledge. This includes fighting giant animals, solving complex puzzles, and understanding ancient texts such as the "Yorikous Sensei" (the book of the ninja).

3. **Naruto's Goals**: Naruto seeks to become a master ninja and help a larger group of warriors in the kingdom, ofte

In [11]:
def process_pdf(pdf_bytes):
    if pdf_bytes is None:
        return None, None, None
    loader = PyMuPDFLoader(pdf_bytes)
    data = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = splitter.split_documents(data)
    embeddings = OllamaEmbeddings(model="deepseek-r1:1.5b")
    vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory="./chroma_db")
    retriever = vectorstore.as_retriever()
    
    return splitter, vectorstore, retriever

In [12]:
def combine_docs(docs):
    return '\n\n'.join(docs.page_content for docs in docs)

In [13]:
def ollama_llm(question, context):
    
    formattted_prompt= f"Question: {question} \n\n Context: {context}"
    response = ollama.chat(
        model="deepseek-r1:1.5b",  
        messages=[
            {"role": "user", "content": formattted_prompt}, 
        ],
    )
    response_context = response["message"]["content"]
    final_answer = re.sub(r'<think>.*?</think>', '', response_context, flags=re.DOTALL).strip() 
    
    return final_answer

In [14]:
#RAP pipeline
def rag_chain(question, splitter, vectorstore, retriever):
    
    retriever_docs = retriever.invoke(question)
    formatted_content = combine_docs(retriever_docs)
    return ollama_llm(question, formatted_content)

In [15]:
def ask_question(pdf_bytes, question):
    splitter, vectorstore, retriever = process_pdf(pdf_bytes)
    if splitter is None:
        return None 
    result = rag_chain(question, splitter, vectorstore, retriever)
    return result

In [16]:
#frontend
interface = gr.Interface(
    fn =ask_question,
    inputs=[
        gr.File(label="Upload PDF"),
        gr.Textbox(label="Whats the Question?")
    ],
    outputs="text",
    title="Ask Question on PDF",
    description="Upload a PDF and ask a question about it using deepseek-1.5B",
)

interface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/var/folders/t5/_n16k1h57djcdx8b901zxprw0000gn/T/ipykernel_45705/3539978160.py:8: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="deepseek-r1:1.5b")
